<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE_CNN_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import shutil
import librosa
import tarfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
tf.__version__

'2.4.1'

In [38]:
DATASET_ID   = '1HOe5sJe_Juf5uib4f-8pPv-Z64lmqQ4X'
BASE_DIR     = '/content/'
DATA_DIR     = 'Crowd-Emotion-Audio/'
FOLDS        = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS     = ['Approval', 'Disapproval', 'Neutral']

Fs           = 44100
INCREMENT    = 11025 #25%

WINDOW_LEN   = 400
FFT_LEN      = 400
HOP_LENGTH   = 200

In [3]:
#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')   

In [4]:
# ------- Comment This if already downloaded -------- #

destination = os.path.join(BASE_DIR, DATA_DIR)
download_data(DATASET_ID, destination)

cleaning already existing files ... ✕
creating data directory ... √
downloading dataset from the repository ... √
extracting the dataset ... √


In [32]:
count = 0
for emotion in EMOTIONS:
    for fold in FOLDS:
        fold_path = os.path.join(BASE_DIR, DATA_DIR, emotion, fold)
        filenames = os.listdir(fold_path)
        for filename in filenames:
            file_path = os.path.join(fold_path, filename)
            data, _ = librosa.load(
                path        = file_path,
                sr          = Fs,
                mono        = True
            )

            n = data.shape[0]
            i = 0
            while (i + Fs) < n:
                x = data[i:(i + Fs)]
                spectrogram = tf.signal.stft(
                   signals        = x,
                   frame_length   = WINDOW_LEN,
                   frame_step     = HOP_LENGTH,
                   fft_length     = FFT_LEN,
                   window_fn      = tf.signal.hamming_window
                )
                
                i = i + INCREMENT

In [34]:
 s = tf.abs(spectrogram)

In [36]:
s.shape

TensorShape([171, 513])